In [36]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from tqdm import tqdm

In [37]:
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

In [50]:
vectorizer = CountVectorizer(lowercase=True, stop_words=ENGLISH_STOP_WORDS, analyzer='word', min_df = 10, max_df=0.1)
X_train = vectorizer.fit_transform(newsgroups_train.data)

In [51]:
tags = 20
n_k=np.zeros(tags)
n_dk=np.zeros((X_train.shape[0], tags))
n_kw=np.zeros((tags, X_train.shape[1]))
doc, word = X_train.nonzero()
z = np.random.choice(tags, len(doc))
    
for i, j, k in zip(doc, word, z):
    n_dk[i, k] += 1
    n_kw[k, j] += 1
    n_k[k] += 1

In [52]:
def lda(n_dk, n_kw, n_k, z, doc, word, alpha, beta):    
    for i in tqdm(range(50)):
        for j in range(len(doc)):
            cur_word = word[j]
            cur_doc = doc[j]
            cur_tag = z[j]
            
            n_dk[cur_doc, cur_tag] -= 1
            n_kw[cur_topic, cur_word] -= 1
            n_k[cur_tag] -= 1
            
            p = (n_dk[cur_doc, :] + alpha) * (n_kw[:, cur_word] + beta[cur_word]) / (n_k + beta.sum())
            z[j] = np.random.choice(np.arange(tags), p=p / p.sum())
            
            n_dk[cur_doc, z[j]] += 1
            n_kw[z[j], cur_word] += 1
            n_k[z[j]] += 1
    return z, n_kw, n_dk, n_k

In [53]:
z,n_kw, n_dk, n_k = lda(n_dk, n_kw, n_k, z, doc, word, 2 * np.ones(20), 2 * np.ones(X_train.shape[1]))

100%|██████████| 50/50 [24:42<00:00, 29.66s/it]


In [54]:
answer = np.argsort(n_kw, axis=1)[:, -10:]
for i in range(20):
    matrix = np.zeros((1, X_train.shape[1]))
    for j in answer[i]:
        matrix[0, j] = 1
    print('{} \t{}'.format(i + 1, '\t'.join(vectorizer.inverse_transform(matrix)[0])))

1 	address	available	code	following	ftp	information	program	send	space	systems
2 	course	information	let	little	long	really	said	sure	tell	thing
3 	10	11	12	14	15	16	18	20	24	25
4 	actually	better	case	course	did	quite	really	sure	work	wrong
5 	banks	cadre	dsl	edu	geb	gordon	pitt	skepticism	soon	surrender
6 	case	control	country	government	gun	law	public	state	states	years
7 	didn	going	got	little	ll	lot	maybe	old	really	saw
8 	children	government	israel	jewish	jews	killed	war	women	world	years
9 	cost	course	great	ll	look	point	real	sure	things	year
10 	able	ago	believe	example	group	guess	kind	ll	probably	sure
11 	best	com	david	difference	ll	post	question	sounds	source	try
12 	actually	doesn	going	interested	ll	look	makes	question	really	stuff
13 	com	guess	interested	look	looking	months	sure	tell	things	turn
14 	chip	clipper	encryption	government	information	key	keys	number	phone	public
15 	card	drive	help	mail	pc	problem	software	using	windows	work
16 	com	did	ll	looking	lot	mail	

In [9]:
>newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

* 1 - comp.os.ms-windows.misc Интеррнет термины
* 6 - talk.politics.guns или talk.politics.misc 
* 8 - talk.politics.mideast Политика и Израиль
* 14 - sci.crypt 
* 15 - comp.sys.mac.hardware Компьютерные термины
* 19 - может быть soc.religion.christian
* 20 - здесь игры, может быть rec.sport.baseball или rec.sport.hockey











